이를 위해 pyspark과 Py4J 패키지를 설치한다. Py4J 패키지는 파이썬 프로그램이 자바가상머신상의 오브젝트들을 접근할 수 있게 해준다. Local Standalone Spark을 사용한다.

In [1]:
!pip install pyspark==3.3.1 py4j==0.10.9.5 

  Using cached pyspark-3.3.1-py2.py3-none-any.whl
  Using cached py4j-0.10.9.5-py2.py3-none-any.whl (199 kB)
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.4.1
    Uninstalling pyspark-3.4.1:
      Successfully uninstalled pyspark-3.4.1


**Spark Session**을 하나 만든다

In [1]:
# 코드를 작성하기 전에 test 코드를 먼저 만들어두고, 코드를 만들어라
# TDD(TEST DRIVEN DEVELOPMENT)

from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark Unit Test") \
    .getOrCreate()

In [3]:
!wget https://s3-geospatial.s3-us-west-2.amazonaws.com/name_gender.csv

'wget'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [2]:
df = spark.read.option("header", True).csv("./data/name_gender.csv")
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- gender: string (nullable = true)



In [3]:
df.count()

100

In [4]:
df.createOrReplaceTempView("namegender")
spark.sql("SELECT gender, COUNT(1) count FROM namegender GROUP BY 1").show()

+------+-----+
|gender|count|
+------+-----+
|     F|   65|
|     M|   28|
|Unisex|    7|
+------+-----+



In [8]:
from pyspark.sql.functions import pandas_udf
from pyspark.sql.types import *
import pandas as pd

# Define the UDF
@pandas_udf(StringType())
def upper_udf_f(s: pd.Series) -> pd.Series:
    return s.str.upper()

upperUDF = spark.udf.register("upper_udf", upper_udf_f)

Py4JError: An error occurred while calling None.org.apache.spark.api.python.PythonFunction. Trace:
py4j.Py4JException: Constructor org.apache.spark.api.python.PythonFunction([class [B, class java.util.HashMap, class java.util.ArrayList, class java.lang.String, class java.lang.String, class java.util.ArrayList, class org.apache.spark.api.python.PythonAccumulatorV2]) does not exist
	at py4j.reflection.ReflectionEngine.getConstructor(ReflectionEngine.java:180)
	at py4j.reflection.ReflectionEngine.getConstructor(ReflectionEngine.java:197)
	at py4j.Gateway.invoke(Gateway.java:237)
	at py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
	at py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
	at py4j.ClientServerConnection.waitForCommands(ClientServerConnection.java:182)
	at py4j.ClientServerConnection.run(ClientServerConnection.java:106)
	at java.base/java.lang.Thread.run(Thread.java:833)



In [6]:
def load_gender(spark, file_path):
    return spark.read.option("header", True).csv(file_path)

def get_gender_count(spark, df, field_to_count):
    df.createOrReplaceTempView("namegender_test")
    return spark.sql(f"SELECT {field_to_count}, COUNT(1) count FROM namegender_test GROUP BY 1")

In [ ]:
df = load_gender(spark, "./data/name_gender.csv")
get_gender_count(spark, df, "gender").show()
df.select(upperUDF("name").alias("NAME")).show()

+------+-----+
|gender|count|
+------+-----+
|     F|   65|
|     M|   28|
|Unisex|    7|
+------+-----+

+----------+
|      NAME|
+----------+
|  ADALEIGH|
|     AMRYN|
|    APURVA|
|    ARYION|
|    ALIXIA|
|ALYSSAROSE|
|    ARVELL|
|     AIBEL|
|   ATIYYAH|
|     ADLIE|
|    ANYELY|
|    AAMONI|
|     AHMAN|
|    ARLANE|
|   ARMONEY|
|   ATZHIRY|
| ANTONETTE|
|   AKEELAH|
| ABDIKADIR|
|    ARINZE|
+----------+
only showing top 20 rows



In [ ]:
df.select(upperUDF("name").alias("NAME")).collect()

[Row(NAME='ADALEIGH'),
 Row(NAME='AMRYN'),
 Row(NAME='APURVA'),
 Row(NAME='ARYION'),
 Row(NAME='ALIXIA'),
 Row(NAME='ALYSSAROSE'),
 Row(NAME='ARVELL'),
 Row(NAME='AIBEL'),
 Row(NAME='ATIYYAH'),
 Row(NAME='ADLIE'),
 Row(NAME='ANYELY'),
 Row(NAME='AAMONI'),
 Row(NAME='AHMAN'),
 Row(NAME='ARLANE'),
 Row(NAME='ARMONEY'),
 Row(NAME='ATZHIRY'),
 Row(NAME='ANTONETTE'),
 Row(NAME='AKEELAH'),
 Row(NAME='ABDIKADIR'),
 Row(NAME='ARINZE'),
 Row(NAME='ARSHAUN'),
 Row(NAME='ALEXANDRO'),
 Row(NAME='AYRIAUNA'),
 Row(NAME='AQIB'),
 Row(NAME='ALLEYA'),
 Row(NAME='AAVAH'),
 Row(NAME='ANESTI'),
 Row(NAME='ADALAIDE'),
 Row(NAME='ANALENA'),
 Row(NAME='ALAEYAH'),
 Row(NAME='ALBENA'),
 Row(NAME='AIMI'),
 Row(NAME='ADWAITH'),
 Row(NAME='ARKADY'),
 Row(NAME='ASTYN'),
 Row(NAME='ADELEE'),
 Row(NAME='AGATA'),
 Row(NAME='ALEGNA'),
 Row(NAME='ALTAN'),
 Row(NAME='AHNALEIGH'),
 Row(NAME='ALGIE'),
 Row(NAME='ASHANTI'),
 Row(NAME='AISLYN'),
 Row(NAME='ADALEINE'),
 Row(NAME='ANTHNOY'),
 Row(NAME='ALGERNON'),
 Row(NAME='

## 유닛 테스트 코드 붙여보기

In [ ]:
from unittest import TestCase

# 일반적으로는 아래 함수가 정의된 모듈을 임포트하고 그걸 테스트
#  - upper_udf_f
#  - load_gender
#  - get_gender_count

# 보통 테스트 코드를 만든 다음에, 내가 테스트 하고 싶은 함수들을 import 해서 쓰는 것이 일반적

# 이 외에도 2가지 방법이 더 존재
# - from pyspark.sql.tests import SparkTestingBase
# - pytest-spark (pytest testing framework plugin)

class UtilsTestCase(TestCase):
    # 전역 변수 생성
    spark = None

    # setUpClass는 계속해서 사용할 리소스들이 있으면, 여기서 생성
    # spark 세션을 하나 만들어서 Spark이라는 변수에 저장을 해놓았다.
    # self.spark으로 지칭
    @classmethod
    def setUpClass(cls) -> None:
        cls.spark = SparkSession.builder \
            .appName("Spark Unit Test") \
            .getOrCreate()

    # 데이터프레임 받아서 그거의 카운트 세서, 100과 같은지 다른지 체크
    # assertEqual은 두 개의 인자가 같으면 성공, 실패
    def test_datafile_loading(self):
        sample_df = load_gender(self.spark, "./data/name_gender.csv")
        result_count = sample_df.count()
        self.assertEqual(result_count, 100, "Record count should be 100")

    # 데이터 프레임 받은 후
    # get gender count해서 콜렉트 해서
    # 파이썬에서 row 타입의 리스트로 받아온 후
    # 숫자가 맞으면
    def test_gender_count(self):
        sample_df = load_gender(self.spark, "./data/name_gender.csv")
        count_list = get_gender_count(self.spark, sample_df, "gender").collect()
        count_dict = dict()
        for row in count_list:
            count_dict[row["gender"]] = row["count"]
        self.assertEqual(count_dict["F"], 65, "Count for F should be 65")
        self.assertEqual(count_dict["M"], 28, "Count for M should be 28")
        self.assertEqual(count_dict["Unisex"], 7, "Count for Unisex should be 7")

    # udf를 테스트해보는 케이스
    # spark 데이터 프레임으로 만들고
    # upper_udf 함수 등록
    # 대문자로 바뀐 컬럼 이름을 그 데이터프레임의 결과를 collect로 받아온다.
    def test_upper_udf(self):
        test_data = [
            { "name": "John Kim" },
            { "name": "Johnny Kim"},
            { "name": "1234" }
        ]
        expected_results = [ "JOHN KIM", "JOHNNY KIM", "1234" ]

        upperUDF = self.spark.udf.register("upper_udf", upper_udf_f)
        test_df = self.spark.createDataFrame(test_data)
        names = test_df.select("name", upperUDF("name").alias("NAME")).collect()
        results = []
        for name in names:
            results.append(name["NAME"])
            
        # assertCountEqual은 리스트를 두개 받아서 리스트를 sorting 했을 대 동일한지 보는 것
        self.assertCountEqual(results, expected_results)

    
    # tearDownClass는 마지막에 호출
    # setUpClass에서 release
    @classmethod
    def tearDownClass(cls) -> None:
        cls.spark.stop()

In [ ]:
import unittest

unittest.main(argv=[''], verbosity=2, exit=False)

test_datafile_loading (__main__.UtilsTestCase) ... ok
test_gender_count (__main__.UtilsTestCase) ... ERROR
test_upper_udf (__main__.UtilsTestCase) ... 

In [ ]:
# cli에서 테스트
